In [168]:
import pandas as pd
import json,natsort
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [169]:
hw_data_file = './combined_uarch.csv'
high_level_data_file = './all_config_stacked.csv'
mapping_file = './mapping_funcs.json'

In [170]:
# read data as csv
hw_df = pd.read_csv(hw_data_file, index_col=0)
high_level_df = pd.read_csv(high_level_data_file)

In [171]:
hw_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,NI_FindObjects,3.035,9.287872e+11,3.633920e+10,0.000000e+00,0.000000e+00,1.816960e+10,0.0,4.606528e+11,0.0,0.0,0.0,0.0,1.389440e+11,2.854187,8
1,__pyx_fuse_2__pyx_f_9_ni_label_fused_read_line,2.015,7.408000e+11,4.148480e+10,1.481600e+09,1.059344e+11,1.481600e+10,0.0,2.844672e+11,0.0,0.0,0.0,0.0,7.408000e+09,1.894955,8
2,__pyx_fuse_2__pyx_f_9_ni_label_fused_write_line,1.770,6.032000e+11,1.568320e+10,4.825600e+09,0.000000e+00,0.000000e+00,0.0,1.749280e+10,0.0,0.0,0.0,0.0,0.000000e+00,1.664551,8
3,__pyx_f_9_ni_label_label_line_with_neighbor,1.410,2.966544e+11,3.924000e+10,7.638720e+10,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,1.109184e+11,1.325998,8
4,__pyx_pw_9_ni_label_7_label,1.315,3.816000e+11,5.952000e+10,6.096000e+10,1.776000e+10,0.000000e+00,0.0,1.766400e+11,0.0,0.0,0.0,0.0,0.000000e+00,1.236658,8


In [172]:
high_level_df.head()

,config,Cast,GaussianNoise,LoadImage,RandBalancedCrop,RandFlip,RandomBrightnessAugmentation
0,8,0.363379,1.085471,12.101996,15.306179,0.738835,0.132086


In [173]:
# make config column the index
high_level_df = high_level_df.set_index('config')

# map config column with format 'b1024_gpu4_dataloader8' to '8' casted to int
# hw_df['config'] = hw_df['config'].map(lambda x: int(x.split('_')[-1].replace('dataloader', '')))

In [174]:
print('Numbers are %s (not multiplied by 100)')
high_level_df.head()

Numbers are %s (not multiplied by 100)


,Cast,GaussianNoise,LoadImage,RandBalancedCrop,RandFlip,RandomBrightnessAugmentation
config,,,,,,
8,0.363379,1.085471,12.101996,15.306179,0.738835,0.132086


In [175]:
hw_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,NI_FindObjects,3.035,9.287872e+11,3.633920e+10,0.000000e+00,0.000000e+00,1.816960e+10,0.0,4.606528e+11,0.0,0.0,0.0,0.0,1.389440e+11,2.854187,8
1,__pyx_fuse_2__pyx_f_9_ni_label_fused_read_line,2.015,7.408000e+11,4.148480e+10,1.481600e+09,1.059344e+11,1.481600e+10,0.0,2.844672e+11,0.0,0.0,0.0,0.0,7.408000e+09,1.894955,8
2,__pyx_fuse_2__pyx_f_9_ni_label_fused_write_line,1.770,6.032000e+11,1.568320e+10,4.825600e+09,0.000000e+00,0.000000e+00,0.0,1.749280e+10,0.0,0.0,0.0,0.0,0.000000e+00,1.664551,8
3,__pyx_f_9_ni_label_label_line_with_neighbor,1.410,2.966544e+11,3.924000e+10,7.638720e+10,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,1.109184e+11,1.325998,8
4,__pyx_pw_9_ni_label_7_label,1.315,3.816000e+11,5.952000e+10,6.096000e+10,1.776000e+10,0.000000e+00,0.0,1.766400e+11,0.0,0.0,0.0,0.0,0.000000e+00,1.236658,8


In [176]:
# load the mapping functions
mapping_funcs = json.load(open(mapping_file))


In [177]:
mapping_funcs

{'op_to_func': {'Cast': ['munmap|libc.so.6',
   '__memmove_avx_unaligned_erms|libc.so.6'],
  'LoadImage': ['__GI__IO_fread|libc.so.6',
   'method_vectorcall_NOARGS|python3.8'],
  'RandFlip': ['_PyDict_LoadGlobal|python3.8',
   '_contig_to_contig|_multiarray_umath.cpython-38-x86_64-linux-gnu.so',
   '_aligned_strided_to_contig_size1|_multiarray_umath.cpython-38-x86_64-linux-gnu.so',
   '[Outside any known module]|[Unknown]',
   '_aligned_strided_to_contig_size4|_multiarray_umath.cpython-38-x86_64-linux-gnu.so',
   '<listcomp>|numeric.py',
   'raw_array_assign_array|_multiarray_umath.cpython-38-x86_64-linux-gnu.so',
   'munmap|libc.so.6',
   '__memmove_avx_unaligned_erms|libc.so.6'],
  'GaussianNoise': ['__log_compat|libm.so.6',
   '__vfprintf|libc-dynamic.so',
   '__pyx_f_5numpy_6random_7_common_cont|_common.cpython-38-x86_64-linux-gnu.so',
   'FLOAT_add_AVX2|_multiarray_umath.cpython-38-x86_64-linux-gnu.so',
   'legacy_normal|mtrand.cpython-38-x86_64-linux-gnu.so',
   '_aligned_contig_

In [178]:
# print columns of hw_df
hw_df.columns

Index(['Function', 'CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local Memory', 'Remote Memory', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %', 'config'],
      dtype='object')

In [179]:
hw_columns_to_su = ['CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local Memory', 'Remote Memory', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %']
combined_df = pd.DataFrame()
# loop config column in high_level_df
for config in high_level_df.index:
    print('Config: ',config)
    config_df = pd.DataFrame()
    # loop through each column in high_level_df
    for python_func in high_level_df.columns:
        print('\tPython func: ',python_func)
        # get the mapping function for the python_func
        c_cppfuncs = mapping_funcs['op_to_func'][python_func]
        python_df = pd.DataFrame()
        # loop through each column in mapping_func
        for c in c_cppfuncs:
            # rename the c func if it exists in the func_rename dict
            func_only = c.split('|')[0]
            print('\t\tC/C++ func: ',func_only)
            # get the column in hw_df that matches the c func and config
            hw_df_col = hw_df[(hw_df['config'] == config) & (hw_df['Function'] == func_only)].copy()
            # check if empty
            if hw_df_col.empty:
                print('\t\t\tEmpty')
                continue
            # print('\t\t\tHW data: ',hw_df_col)
            # drop the config and function column
            hw_df_col = hw_df_col.drop(columns=['config', 'Function'])
            # multiply the hw_df_col by the percentage of the python_func if its a common op
            if c in mapping_funcs['common_funcs_to_op']:
                ops = mapping_funcs['common_funcs_to_op'][c]
                base_total = 0
                for op in ops:
                    base_total += high_level_df.loc[config, op]
                scale_factor = high_level_df.loc[config, python_func] / base_total
                print('\t\t\tScale factor: ',scale_factor)
                hw_df_col = hw_df_col.mul(scale_factor)
            # concat the hw_df_col to the python_df
            python_df = pd.concat([python_df, hw_df_col])
        # sum the python_df and print
        python_series = python_df.sum()
        python_df = pd.DataFrame({python_func: python_series})
        # concat the python_df to the config_df
        config_df = pd.concat([config_df, python_df],axis=1)

    # transpose the config_df
    config_df = config_df.T
    # make index a column with name 'Function'
    config_df = config_df.reset_index().rename(columns={'index': 'Function'})
    # add config column to config_df
    config_df['config'] = config
    # concat the config_df to the combined_df
    combined_df = pd.concat([combined_df, config_df])
# reset the index
combined_df = combined_df.reset_index(drop=True)

Config:  8
	Python func:  Cast
		C/C++ func:  munmap
			Empty
		C/C++ func:  __memmove_avx_unaligned_erms
			Scale factor:  0.02214592903712431
	Python func:  GaussianNoise
		C/C++ func:  __log_compat
			Empty
		C/C++ func:  __vfprintf
			Empty
		C/C++ func:  __pyx_f_5numpy_6random_7_common_cont
		C/C++ func:  FLOAT_add_AVX2
		C/C++ func:  legacy_normal
		C/C++ func:  _aligned_contig_cast_double_to_float
			Scale factor:  0.8915155434882325
		C/C++ func:  __pyx_f_5numpy_6random_8_mt19937_mt19937_double
		C/C++ func:  func@0xc1d0
		C/C++ func:  munmap
			Empty
		C/C++ func:  legacy_gauss
		C/C++ func:  mt19937_gen
		C/C++ func:  __ieee754_log_fma
	Python func:  LoadImage
		C/C++ func:  __GI__IO_fread
			Empty
		C/C++ func:  method_vectorcall_NOARGS
	Python func:  RandBalancedCrop
		C/C++ func:  __pyx_fuse_2__pyx_f_9_ni_label_fused_read_line
		C/C++ func:  UBYTE_greater_AVX2
		C/C++ func:  __pyx_f_9_ni_label_label_line_with_neighbor
		C/C++ func:  array_boolean_subscript
		C/C++ func:  P

In [180]:
combined_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,Cast,0.012402,5.835895e+08,5.692390e+08,2.965783e+08,1.272416e+09,5.453214e+08,0.000000e+00,3.410650e+09,0.000000e+00,0.0,0.0,4.783521e+09,0.000000e+00,0.011663,8
1,GaussianNoise,1.064033,2.632100e+11,4.928000e+09,5.257600e+10,3.642240e+10,3.163219e+10,0.000000e+00,1.854352e+10,0.000000e+00,0.0,0.0,0.000000e+00,7.420073e+10,1.000643,8
2,LoadImage,0.015000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.014106,8
3,RandBalancedCrop,12.542383,3.333262e+12,2.560302e+11,2.086684e+11,3.350893e+11,1.041523e+11,0.000000e+00,1.651591e+12,2.208000e+11,0.0,0.0,2.384713e+11,2.623392e+11,11.795159,8
4,RandFlip,0.310216,3.391617e+10,1.157396e+09,6.030132e+08,2.587121e+09,1.777437e+10,1.929440e+10,8.053465e+10,0.000000e+00,0.0,0.0,9.726019e+09,3.400000e+10,0.291734,8


In [181]:
# print columns of combined_df
combined_df.columns

Index(['Function', 'CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local Memory', 'Remote Memory', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %', 'config'],
      dtype='object')

In [182]:
combined_df.head(len(combined_df))

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local Memory,Remote Memory,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,Cast,0.012402,5.835895e+08,5.692390e+08,2.965783e+08,1.272416e+09,5.453214e+08,0.000000e+00,3.410650e+09,0.000000e+00,0.0,0.0,4.783521e+09,0.000000e+00,0.011663,8
1,GaussianNoise,1.064033,2.632100e+11,4.928000e+09,5.257600e+10,3.642240e+10,3.163219e+10,0.000000e+00,1.854352e+10,0.000000e+00,0.0,0.0,0.000000e+00,7.420073e+10,1.000643,8
2,LoadImage,0.015000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.014106,8
3,RandBalancedCrop,12.542383,3.333262e+12,2.560302e+11,2.086684e+11,3.350893e+11,1.041523e+11,0.000000e+00,1.651591e+12,2.208000e+11,0.0,0.0,2.384713e+11,2.623392e+11,11.795159,8
4,RandFlip,0.310216,3.391617e+10,1.157396e+09,6.030132e+08,2.587121e+09,1.777437e+10,1.929440e+10,8.053465e+10,0.000000e+00,0.0,0.0,9.726019e+09,3.400000e+10,0.291734,8
5,RandomBrightnessAugmentation,0.070967,7.356114e+08,0.000000e+00,0.000000e+00,9.600000e+09,1.902210e+09,0.000000e+00,2.256477e+09,0.000000e+00,0.0,0.0,0.000000e+00,2.500065e+09,0.066739,8


In [183]:
for col in combined_df.columns:
    if col == 'config' or col == 'Function':
        continue
    print(col)
    # pivot table
    df_pivot = combined_df.pivot(index='config', columns='Function', values=col)
    # sort the index using natsort
    df_pivot = df_pivot.reindex(natsort.natsorted(df_pivot.index))
    # find lowest value in entire dataframe and normalize the dataframe by dividing each value by the lowest value suhc that its not 0
    ax = df_pivot.plot(kind='bar', stacked=True, figsize=(12,6),colormap='jet')
    # for c in ax.containers:
    #     ax.bar_label(c, label_type='center')
    # make x axis label horizontal
    plt.xticks(rotation=0)
    # add x axis label
    plt.xlabel('Dataloaders')
    # add y axis label
    plt.ylabel(f"{col}")
    # add title to legend
    plt.title(f"Hardware metric for preprocessing operations")    
    # reverse legend order
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles[::-1], labels[::-1],loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.subplots_adjust(right=0.70)
    plt.savefig(f'./mapped_funcs_metric_figs/{col}.png')
    plt.close()


CPU Time (s)
Retiring
Front-End Bound
Bad Speculation
L1 Bound
L2 Bound
L3 Bound
Memory Bandwidth
Local Memory
Remote Memory
Remote Cache
Store Bound
Core Bound
CPU Time %
